In [1]:
import ee
ee.Initialize()

In [2]:
import geemap
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import shapely as shp
import os
import re
import string
import math

In [10]:
def geometryToEE(img_geometry):
    
    # format geometry
#     img_geometry = gpd.GeoDataFrame(geometry = img_geometry)
    print(img_geometry)
    img_geometry = img_geometry.reset_index(drop = True)
    img_geometry = [[[x, y] for x, y in list(img_geometry.geometry.exterior[0].coords)]]
    
    # convert geometry to ee.Geometry
    img_geometry_ee = ee.Geometry({
        'type': 'Polygon',
        'coordinates': img_geometry
    })
    
    return img_geometry_ee

In [35]:
# Read in RTS test tiles
tiles = gpd.read_file('../data/rts_polygons/all_tiles.shp')
tiles = tiles.to_crs(4326)

aoi = tiles.geometry.bounds.iloc[0]
print(aoi)
aoi = ee.Geometry({
        'type': 'Polygon',
        'coordinates': [
            [
                [aoi[0], aoi[2]],
                [aoi[1], aoi[2]],
                [aoi[1], aoi[3]],
                [aoi[0], aoi[3]],
                [aoi[0], aoi[2]]
                        ]
        ]
    })
print(aoi.getInfo())

tiles

minx    68.026573
miny    70.781306
maxx    68.042567
maxy    70.787515
Name: 0, dtype: float64
{'type': 'Polygon', 'coordinates': [[[68.02657262962772, 68.04256671230145], [70.78130569327571, 68.04256671230145], [70.78130569327571, 70.7875145426991], [68.02657262962772, 70.7875145426991], [68.02657262962772, 68.04256671230145]]]}


,id,imagery,geometry
0,00000000000000000002,WorldView,"POLYGON ((68.03283 70.78751, 68.04257 70.78615..."
1,00000000000000000006,WorldView,"POLYGON ((67.84626 70.71843, 67.85593 70.71709..."
2,00000000000000000007,WorldView,"POLYGON ((67.84569 70.71748, 67.85536 70.71613..."
3,00000000000000000008,WorldView,"POLYGON ((67.91335 70.62325, 67.92301 70.62190..."
4,0000000000000000000a,WorldView,"POLYGON ((67.91373 70.62254, 67.92344 70.62117..."
...,...,...,...
184,0000000000000000007f,Sentinel-2,"POLYGON ((-133.94877 69.05770, -133.95407 69.0..."
185,00000000000000000081,Sentinel-2,"POLYGON ((-138.86547 69.56958, -138.87145 69.5..."
186,00000000000000000084,Sentinel-2,"POLYGON ((-139.16281 69.54018, -139.16889 69.5..."
187,00000000000000000086,Sentinel-2,"POLYGON ((-139.21393 69.55217, -139.22001 69.5..."


In [30]:
# Prep Map
Map = geemap.Map()
Map.centerObject(aoi);

In [31]:
# Get Sentinel-2 Mosaic to Align Planet Data
MAX_CLOUD_PROBABILITY = 50
# geometry = ee.Geometry({
#     'type': 'Polygon',
#     'coordinates': [[
#         [66, 66],
#         [66, 73.5],
#         [85, 73.5],
#         [85, 66],
#         [66, 66]
#         ]]
# })

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

s2_clouds = (
    ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

def s2_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2_with_clouds = ee.Join.saveFirst('cloud_mask').apply(    
  primary=s2,
  secondary=s2_clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

s2_masked = ee.ImageCollection(s2_with_clouds).map(s2_maskcloud)

# s2_masked_north = s2_masked.filter(ee.Filter.dayOfYear(196, 243))

def s2_water_mask(image):
    return image.select('SCL').eq(6)

s2_water = (
    s2_masked.map(s2_water_mask)
    .median()
);

s2_mosaic = (
    s2_masked
    .select(['B2', 'B3', 'B4', 'B8'])
    .median());

# s2_mosaic_north = (
#     s2_masked_north
#     .select(['B2', 'B3', 'B4', 'B8'])
#     .median());

pprint(s2_water.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 1,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'SCL'}],
 'type': 'Image'}


In [32]:
Map.addLayer(s2_mosaic.clip(aoi), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1000}, 
             'S2 All Years')
Map.addLayer(s2_water.clip(aoi), 
             {'min':0, 'max':1}, 
             'S2 Water')


In [33]:
Map

Map(center=[69.41761287493608, 69.40393916145172], controls=(WidgetControl(options=['position', 'transparent_b…

In [44]:
for index, row in tiles.iterrows():
    polygon_id = row.id
    name = 'Sentinel-2_water_mask_' + str(row.id)
    img_geometry = [[[x, y] for x, y in list(row.geometry.exterior.coords)]]
    crs = 'EPSG:' + str(3413)
   
    # Export 2018-2019 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toDrive(
        image = s2_water,
        description = name,
        folder = 'Earth Engine Exports',
        crs = crs,
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF'
    )
    task.start()